## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [391]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [392]:

## loading the debar dataset

debar = df = pd.read_csv("debar.csv")

## viewing the head of debar

debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [393]:

## printing the number of rows

print(debar.shape[0])

## printing the number of unique employer names

print(debar.Name.nunique())

## Question 1: Is there one row per employer or multiple rows for some employers? 

# There are 114 rows in the dataset, and 98 unique employer names. Consequently, I can conclude that there are mulitple rows for some employers.


114
98


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [394]:

## creating a new column — is_repeated

repeat = debar["Name"].value_counts()

debar["is_repeated"] = debar['Name'].apply(lambda x: repeat[x] > 1)

## printing the rows where is_repeated = True

print(debar[debar["is_repeated"] == True].shape[0])

## interpreting the print

# There are 32 instances where the name of the employer repeats more than one time. 

## subsetting to rows where is_repeated is True

mult_debar = debar[debar["is_repeated"] == True].copy()

## printing the head and shape of mult_debar

print(mult_debar.head())
print(mult_debar.shape)


32
                                     Name      City, State  \
6                 Annabella Land & Cattle    Annabella, UT   
7                     Autumn Hill Orchard       Groton, MA   
8   Caddo Creek Ranch, dba Paradise Ranch        Caddo, TX   
11                  Loewen Harvesting LLC  Brownsville, TX   
12            Rollo Farm Labor Contractor        Miami, FL   

                                         Violation Duration Start date  \
6                                      Non Payment   1 year   5/9/2014   
7        Failure to respond to audit (no response)  2 years   7/6/2014   
8   Failure to respond to audit (partial response)  2 years  7/20/2014   
11  Failure to respond to audit (partial response)   1 year  8/20/2014   
12       Failure to respond to audit (no response)  2 years  8/23/2014   

     End date  is_repeated  
6    5/9/2015         True  
7    7/5/2016         True  
8   7/19/2016         True  
11  8/19/2015         True  
12  8/22/2016         True  
(32, 

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [395]:

## fixing the 'Start Date' column

mult_debar["Start Date"] = pd.to_datetime(mult_debar['Start date'], format = 'mixed')

## creating a cumulative count column 

mult_debar["cumulcounts"] = mult_debar.groupby("Name").cumcount()

## adjusting 'City,State' when is_repeated = True

mult_debar['City,State_STR'] = mult_debar['City, State'].astype(str)
mult_debar['City,State_STR'] = mult_debar['City,State_STR'].str.replace('Utah', 'UT')
mult_debar['City,State_STR'] = mult_debar['City,State_STR'].str.replace('Georgia', 'GA')
mult_debar['City,State_STR'] = mult_debar['City,State_STR'].str.replace('Texas', 'TX')
mult_debar['City,State_STR'] = mult_debar['City,State_STR'].str.replace('AK', 'AR')

## creating a str column

mult_debar["strcounts"] = mult_debar["cumulcounts"].apply(lambda x: "start_date_viol" + str(x + 1))

## reshaping mult_debar to wide-form

mult_debar_wide = mult_debar.pivot(index = ['Name', 'City,State_STR'], columns = 'strcounts', values = 'Start Date')

## printing the head and shape of mult_debar_wide

mult_debar_wide.head()

mult_debar_wide.shape


,strcounts,start_date_viol1,start_date_viol2
Name,"City,State_STR",,
Annabella Land & Cattle,"Annabella, UT",2014-05-09,2014-05-09
Autumn Hill Orchard,"Groton, MA",2014-07-06,2014-07-06
"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",2014-07-20,2014-07-20
Cisco Produce Inc.,"Cairo, GA",2014-12-10,2015-12-10
Dove Creek Farms,"Mount Vernon, TX",2018-02-09,2018-02-09


(17, 2)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [ ]:
# FIX THIS ONE

In [396]:

## creating the column is_dup

mult_debar_wide["is_dup"] = (mult_debar_wide["start_date_viol1"] == mult_debar_wide["start_date_viol2"])

## filtering to employers where is_dup = True
employ_true = mult_debar_wide[mult_debar_wide["is_dup"] == True]

employ_true

list = ["Annabella Land & Cattle", "Autumn Hill Orchard", "Caddo Creek Ranch, dba Paradise Ranch", "Dove Creek Farms", 
       "F&W Farms", "Macky and Brad Farms", "Maple Ridge Custom Services, LLC", "Mark Duncan", "Rollo Farm Labor Contractor", 
       "SRT Farms", "Sharon Mathis", "Turner Farms"]

## creating a violnum column 

mult_debar["violnum"] = mult_debar["strcounts"].str.replace('start_date_', '')
mult_debar

## filtering to keep only violnum = viol1

filtered_1 = mult_debar[(mult_debar['Name'].isin(list)) & (mult_debar['violnum'] == 'viol1')]

## filtering to employers where is_dup = False

employ_false = mult_debar_wide[mult_debar_wide["is_dup"] == False]

employ_false

list_2 = ["Cisco Produce Inc.", "Loewen Harvesting LLC", "Old Tree Farms/Verpaalen Custom Service", "Xavier Horne"]

## filtering to keep all violnum

filtered_2 = mult_debar[(mult_debar['Name'].isin(list_2)) & (mult_debar['violnum'])]

filtered_2

## removing is_repeated from the debar data

debar_update = mult_debar.drop(columns = ['is_repeated'], inplace = True)

debar_update

## concatenating to make debar_clean

debar_clean = pd.concat([filtered_1, filtered_2, debar_update], axis = 0)

## printing debar_clean shape and head

print(debar_clean.head())
print(debar_clean.shape)

debar_clean["Name"].shape


,strcounts,start_date_viol1,start_date_viol2,is_dup
Name,"City,State_STR",,,
Annabella Land & Cattle,"Annabella, UT",2014-05-09,2014-05-09,True
Autumn Hill Orchard,"Groton, MA",2014-07-06,2014-07-06,True
"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",2014-07-20,2014-07-20,True
Dove Creek Farms,"Mount Vernon, TX",2018-02-09,2018-02-09,True
F&W Farms,"Ingalls, KS",2014-12-10,2014-12-10,True
Macky and Brad Farms,"Plains, TX",2015-02-13,2015-02-13,True
"Maple Ridge Custom Services, LLC","Altheimer, AR",2014-11-16,2014-11-16,True
Mark Duncan,"Roosevelt, UT",2014-11-16,2014-11-16,True
Rollo Farm Labor Contractor,"Miami, FL",2014-08-23,2014-08-23,True


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,Start Date,cumulcounts,"City,State_STR",strcounts,violnum
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True,2014-05-09,0,"Annabella, UT",start_date_viol1,viol1
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True,2014-07-06,0,"Groton, MA",start_date_viol1,viol1
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True,2014-07-20,0,"Caddo, TX",start_date_viol1,viol1
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True,2014-08-20,0,"Brownsville, TX",start_date_viol1,viol1
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True,2014-08-23,0,"Miami, FL",start_date_viol1,viol1
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,2014-11-16,0,"Tifton, GA",start_date_viol1,viol1
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,2014-11-16,0,"Morton, TX",start_date_viol1,viol1
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True,2014-11-16,0,"Roosevelt, UT",start_date_viol1,viol1
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True,2014-11-16,0,"Altheimer, AR",start_date_viol1,viol1
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True,2014-12-10,0,"Ingalls, KS",start_date_viol1,viol1


strcounts                                               start_date_viol1  \
Name                                    City,State_STR                     
Cisco Produce Inc.                      Cairo, GA             2014-12-10   
Loewen Harvesting LLC                   Brownfield, TX               NaT   
                                        Brownsville, TX       2014-08-20   
Old Tree Farms/Verpaalen Custom Service Volga, SD             2014-12-11   
Xavier Horne                            Lyons, GA             2016-06-16   

strcounts                                               start_date_viol2  \
Name                                    City,State_STR                     
Cisco Produce Inc.                      Cairo, GA             2015-12-10   
Loewen Harvesting LLC                   Brownfield, TX        2014-08-20   
                                        Brownsville, TX              NaT   
Old Tree Farms/Verpaalen Custom Service Volga, SD             2014-12-01   
Xavier Horne                            Lyons, GA             2017-09-27   

strcounts                                                is_dup  
Name                                    City,State_STR           
Cisco Produce Inc.                      Cairo, GA         False  
Loewen Harvesting LLC                   Brownfield, TX    False  
                                        Brownsville, TX   False  
Old Tree Farms/Verpaalen Custom Service Volga, SD         False  
Xavier Horne                            Lyons, GA         False

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,Start Date,cumulcounts,"City,State_STR",strcounts,violnum
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True,2014-08-20,0,"Brownsville, TX",start_date_viol1,viol1
19,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,True,2014-12-10,0,"Cairo, GA",start_date_viol1,viol1
21,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",WHD Debarment,3 years,12/11/2014,12/10/2017,True,2014-12-11,0,"Volga, SD",start_date_viol1,viol1
25,Loewen Harvesting LLC,"Brownfield, TX",Impeding the Audit Process – Partial- Response,1 year,8/20/2014,8/19/2015,True,2014-08-20,1,"Brownfield, TX",start_date_viol2,viol2
51,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",Wage Hour Debarment,3 years,12/1/2014,12/1/2017,True,2014-12-01,1,"Volga, SD",start_date_viol2,viol2
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,True,2015-12-10,1,"Cairo, GA",start_date_viol2,viol2
73,Xavier Horne,"Lyons, Georgia",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,True,2016-06-16,0,"Lyons, GA",start_date_viol1,viol1
89,Xavier Horne,"Lyons, Georgia",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,True,2017-09-27,1,"Lyons, GA",start_date_viol2,viol2


                                     Name    City, State  \
6                 Annabella Land & Cattle  Annabella, UT   
7                     Autumn Hill Orchard     Groton, MA   
8   Caddo Creek Ranch, dba Paradise Ranch      Caddo, TX   
12            Rollo Farm Labor Contractor      Miami, FL   
14                          Sharon Mathis     Tifton, GA   

                                         Violation Duration  Start date  \
6                                      Non Payment   1 year    5/9/2014   
7        Failure to respond to audit (no response)  2 years    7/6/2014   
8   Failure to respond to audit (partial response)  2 years   7/20/2014   
12       Failure to respond to audit (no response)  2 years   8/23/2014   
14       Failure to respond to audit (no response)  2 years  11/16/2014   

      End date  is_repeated Start Date  cumulcounts City,State_STR  \
6     5/9/2015         True 2014-05-09            0  Annabella, UT   
7     7/5/2016         True 2014-07-06          

(20,)

# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [397]:

## loading jobs_clean.csv

jobs_clean = pd.read_csv("jobs.csv")


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [ ]:
# Group Question: Dimensions here? He said it was supposed to be low, but I don't think it shoudl be 0 lol 

In [398]:

## A: inner join

joined = pd.merge(jobs_clean, debar_clean, left_on = "EMPLOYER_NAME", right_on = "Name", how = 'inner', indicator = True)

## B: printing rows with exact matches

exact_match = joined[joined["_merge"] == 'both']

print(exact_match)


Empty DataFrame
Columns: [CASE_NUMBER, CASE_STATUS, RECEIVED_DATE, DECISION_DATE, TYPE_OF_EMPLOYER_APPLICATION, H2A_LABOR_CONTRACTOR, NATURE_OF_TEMPORARY_NEED, EMERGENCY_FILING, EMPLOYER_NAME, TRADE_NAME_DBA, EMPLOYER_ADDRESS_1, EMPLOYER_ADDRESS_2, EMPLOYER_CITY, EMPLOYER_STATE, EMPLOYER_POSTAL_CODE, EMPLOYER_COUNTRY, EMPLOYER_PROVINCE, EMPLOYER_PHONE, EMPLOYER_PHONE_EXT, NAICS_CODE, EMPLOYER_POC_LAST_NAME, EMPLOYER_POC_FIRST_NAME, EMPLOYER_POC_MIDDLE_NAME, EMPLOYER_POC_JOB_TITLE, EMPLOYER_POC_ADDRESS1, EMPLOYER_POC_ADDRESS2, EMPLOYER_POC_CITY, EMPLOYER_POC_STATE, EMPLOYER_POC_POSTAL_CODE, EMPLOYER_POC_COUNTRY, EMPLOYER_POC_PROVINCE, EMPLOYER_POC_PHONE, EMPLOYER_POC_PHONE_EXT, EMPLOYER_POC_EMAIL, TYPE_OF_REPRESENTATION, ATTORNEY_AGENT_LAST_NAME, ATTORNEY_AGENT_FIRST_NAME, ATTORNEY_AGENT_MIDDLE_NAME, ATTORNEY_AGENT_ADDRESS_1, ATTORNEY_AGENT_ADDRESS_2, ATTORNEY_AGENT_CITY, ATTORNEY_AGENT_STATE, ATTORNEY_AGENT_POSTAL_CODE, ATTORNEY_AGENT_COUNTRY, ATTORNEY_AGENT_PROVINCE, ATTORNEY_AGENT_PH

## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [ ]:
## OFFICE HOURS QUESTION: Is this an okay approach for the initial step? 

In [399]:

## A: converting to uppercase using list comprehension

EMPLOYER_NAME_upper = [name.upper() for name in jobs_clean['EMPLOYER_NAME']]

Name_upper = [name.upper() for name in debar_clean['Name']]


In [400]:
## B: printing a random sample of 15 values (jobs_clean dataset)

random.sample(EMPLOYER_NAME_upper, 15)

## printing a random sample of 15 values (debar_clean dataset)

random.sample(Name_upper, 15)


['KAECHELE RANCH LP',
 'BRADLEY LABOR GROUP, LLC',
 'HEARTLAND FARMS, INC.',
 'WESTERN RANGE ASSOCIATION',
 'WENCK BY-PRODUCT MANAGEMENT LLC ',
 'NATURAL PRAIRIE DAIRY - INDIANA, LLC',
 'COASTAL FARM LABOR SERVICES, INC.',
 'VANWINKLE RANCH, LLC',
 'FOUR DEVILLE FARMS, LLC',
 'MANZANA LLC',
 'CHACON SHEEP COMPANY, LLC',
 'DOUBLE J HARVESTING, INC.',
 'ROCKY CREEK NURSERIES INC.',
 'OVERLOOK HARVESTING COMPANY, LLC',
 'COLORFUL GARDENS WHOLESALE LLC']

['DOVE CREEK FARMS',
 'CADDO CREEK RANCH, DBA PARADISE RANCH',
 'XAVIER HORNE',
 'F&W FARMS',
 'MARK DUNCAN',
 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE',
 'LOEWEN HARVESTING LLC',
 'MAPLE RIDGE CUSTOM SERVICES, LLC',
 'CISCO PRODUCE INC.',
 'ANNABELLA LAND & CATTLE',
 'LOEWEN HARVESTING LLC',
 'SRT FARMS',
 'AUTUMN HILL ORCHARD',
 'MACKY AND BRAD FARMS',
 'TURNER FARMS']

In [401]:

## C: writing over the 'EMPLOYER_NAME' column in jobs_clean

jobs_clean['EMPLOYER_NAME'] = EMPLOYER_NAME_upper

debar_clean['Name'] = Name_upper


### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [402]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [403]:
## creating a regex pattern to replace the '.'

regex_pattern = r'(LLC|INC|CO)\.'

regex_replacement = r'\1'

## testing the pattern (positive examples)

test_pos_1 = re.sub(regex_pattern, regex_replacement, pos_example_1)
print(test_pos_1)

test_pos_2 = re.sub(regex_pattern, regex_replacement, pos_example_2)
print(test_pos_2)

## testing the pattern (negative examples)

test_neg = re.sub(regex_pattern, regex_replacement, neg_example)
print(test_neg)


CISCO PRODUCE INC
AVOYELLES HONEY CO, LLC
E.V. RANCH LLP


### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [405]:

## applying the pattern using re.sub

debar_clean['name_clean'] = [re.sub(regex_pattern, r'\1', name) 
                    for name in debar_clean['Name']]
jobs_clean['name_clean'] = [re.sub(regex_pattern, r'\1', name) 
                    for name in jobs_clean['EMPLOYER_NAME']]

## subsetting to rows that changed names

mismatched_debar = debar_clean[debar_clean['Name'] != debar_clean['name_clean']]
mismatched_debar[['Name','name_clean']]
mismatched_jobs = jobs_clean[jobs_clean['EMPLOYER_NAME'] != jobs_clean['name_clean']]
mismatched_jobs[['EMPLOYER_NAME','name_clean']]

## printing the old and updated columns 

debar_clean[["Name","name_clean"]]

jobs_clean[["EMPLOYER_NAME", "name_clean"]]


,Name,name_clean
19,CISCO PRODUCE INC.,CISCO PRODUCE INC
56,CISCO PRODUCE INC.,CISCO PRODUCE INC


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"
...,...,...
2700,HARRAL LIVESTOCK CO. LLC,HARRAL LIVESTOCK CO LLC
2701,ECOSYSTEM CONCEPTS INC.,ECOSYSTEM CONCEPTS INC
2702,SIDDOWAY SHEEP CO.,SIDDOWAY SHEEP CO
2705,SATHER MANAGEMENT INC.,SATHER MANAGEMENT INC


,Name,name_clean
6,ANNABELLA LAND & CATTLE,ANNABELLA LAND & CATTLE
7,AUTUMN HILL ORCHARD,AUTUMN HILL ORCHARD
8,"CADDO CREEK RANCH, DBA PARADISE RANCH","CADDO CREEK RANCH, DBA PARADISE RANCH"
12,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR
14,SHARON MATHIS,SHARON MATHIS
15,SRT FARMS,SRT FARMS
16,MARK DUNCAN,MARK DUNCAN
17,"MAPLE RIDGE CUSTOM SERVICES, LLC","MAPLE RIDGE CUSTOM SERVICES, LLC"
18,F&W FARMS,F&W FARMS
31,MACKY AND BRAD FARMS,MACKY AND BRAD FARMS


,EMPLOYER_NAME,name_clean
0,"FAZIO FARMS OPERATING COMPANY, LLC","FAZIO FARMS OPERATING COMPANY, LLC"
1,CHARLIE SUNDERLAND,CHARLIE SUNDERLAND
2,MICHAEL RUDEBUSCH,MICHAEL RUDEBUSCH
3,LODAHL FARMS,LODAHL FARMS
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
...,...,...
2715,JAMES L SCHNELLER,JAMES L SCHNELLER
2716,STEPHEN G MYERS,STEPHEN G MYERS
2717,ANDY POVEY,ANDY POVEY
2718,"SILVER CREEK SEED, LLC","SILVER CREEK SEED, LLC"


In [407]:
## your code here to print the head

jobs_clean["name_clean"].head()
debar_clean["name_clean"].head()


0    FAZIO FARMS OPERATING COMPANY, LLC
1                    CHARLIE SUNDERLAND
2                     MICHAEL RUDEBUSCH
3                          LODAHL FARMS
4                DUNSON HARVESTING, INC
Name: name_clean, dtype: object

6                   ANNABELLA LAND & CATTLE
7                       AUTUMN HILL ORCHARD
8     CADDO CREEK RANCH, DBA PARADISE RANCH
12              ROLLO FARM LABOR CONTRACTOR
14                            SHARON MATHIS
Name: name_clean, dtype: object

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [301]:
# Group Meeting Issue: Dimensions of joined_2 and joined_3. Also issue with Slash EV Ranch. In general, merge has gone wrong! 

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Violation,Duration,Start date,End date,is_repeated,Start Date,cumulcounts,"City,State_STR",strcounts,violnum


In [408]:
## A: inner join

joined_2 = pd.merge(jobs_clean, debar_clean, left_on = "name_clean", right_on = "name_clean", how = 'inner')

## Question: Did the cleaning result in any more employers matched between the two datasets? 
# Yes, the cleaning resulted in more employers being matched between the datasets. X more employers were matched after the dataset was cleaned.

## B: creating a regex pattern that removes periods between everything and also LLP in general 

regex_pattern_3 = r'\bLLP\b'
#removes LLP

regex_replacement_3 = ''
#indicates to leave blank (not inserting something else in its stead)

regex_pattern_4 = r'\.'
#removes periods 

regex_replacement_4 = ''
#indicates to just keep EV

def fix_name(name_clean):
    return re.sub(regex_pattern_3, regex_replacement_3, name_clean)

def fix_name_2(name_clean):
    return re.sub(regex_pattern_4, regex_replacement_4, name_clean)

## creating a new column — name_clean_2 

debar_clean["name_clean"] = debar_clean["name_clean"].apply(fix_name)
debar_clean["name_clean_2"] = debar_clean["name_clean"].apply(fix_name_2)

## C: conducting a left join using name_clean_2 

joined_3 = pd.merge(jobs_clean, debar_clean, left_on = "name_clean", right_on = "name_clean_2", how = "left")

## print the rows where the merge indicator indicates the row found in both dataframes 

joined_indic = pd.merge(jobs_clean, debar_clean, left_on = "name_clean", right_on = "name_clean_2", how = "left", indicator = True)

matches = joined_indic[joined_indic["_merge"] == 'both']

print(matches)


Empty DataFrame
Columns: [CASE_NUMBER, CASE_STATUS, RECEIVED_DATE, DECISION_DATE, TYPE_OF_EMPLOYER_APPLICATION, H2A_LABOR_CONTRACTOR, NATURE_OF_TEMPORARY_NEED, EMERGENCY_FILING, EMPLOYER_NAME, TRADE_NAME_DBA, EMPLOYER_ADDRESS_1, EMPLOYER_ADDRESS_2, EMPLOYER_CITY, EMPLOYER_STATE, EMPLOYER_POSTAL_CODE, EMPLOYER_COUNTRY, EMPLOYER_PROVINCE, EMPLOYER_PHONE, EMPLOYER_PHONE_EXT, NAICS_CODE, EMPLOYER_POC_LAST_NAME, EMPLOYER_POC_FIRST_NAME, EMPLOYER_POC_MIDDLE_NAME, EMPLOYER_POC_JOB_TITLE, EMPLOYER_POC_ADDRESS1, EMPLOYER_POC_ADDRESS2, EMPLOYER_POC_CITY, EMPLOYER_POC_STATE, EMPLOYER_POC_POSTAL_CODE, EMPLOYER_POC_COUNTRY, EMPLOYER_POC_PROVINCE, EMPLOYER_POC_PHONE, EMPLOYER_POC_PHONE_EXT, EMPLOYER_POC_EMAIL, TYPE_OF_REPRESENTATION, ATTORNEY_AGENT_LAST_NAME, ATTORNEY_AGENT_FIRST_NAME, ATTORNEY_AGENT_MIDDLE_NAME, ATTORNEY_AGENT_ADDRESS_1, ATTORNEY_AGENT_ADDRESS_2, ATTORNEY_AGENT_CITY, ATTORNEY_AGENT_STATE, ATTORNEY_AGENT_POSTAL_CODE, ATTORNEY_AGENT_COUNTRY, ATTORNEY_AGENT_PROVINCE, ATTORNEY_AGENT_PH

# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [215]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here